<img src="Logo.png" width="100" align="left"/> 

# <center> Unit 3 Project </center>
#  <center> Third section : supervised task </center>

In this notebook you will be building and training a supervised learning model to classify your data.

For this task we will be using another classification model "The random forests" model.

Steps for this task: 
1. Load the already clustered dataset 
2. Take into consideration that in this task we will not be using the already added column "Cluster" 
3. Split your data.
3. Build your model using the SKlearn RandomForestClassifier class 
4. classify your data and test the performance of your model 
5. Evaluate the model ( accepted models should have at least an accuracy of 86%). Play with hyper parameters and provide a report about that.
6. Provide evidence on the quality of your model (not overfitted good metrics)
7. Create a new test dataset that contains the testset + an additional column called "predicted_class" stating the class predicted by your random forest classifier for each data point of the test set.

## 1. Load the data and split the data:

In [1]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd 
from sklearn.model_selection import train_test_split

In [8]:
# To-Do:  load the data 
df = pd.read_csv("outputs/unsupervised.csv")
df.head()

,ID,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT,cluster
0,1,0.0,32,1.0,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0,3
1,2,0.0,32,1.0,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5,3
2,3,0.0,32,1.0,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3,3
3,4,0.0,32,1.0,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7,3
4,5,0.0,32,1.0,39.2,74.1,32.6,24.8,9.6,9.15,4.32,76.0,29.9,68.7,3


In [69]:
# To-Do : keep only the columns to be used : all features except ID, cluster 
# The target here is the Category column 
# Do not forget to split your data (this is a classification task)
# test set size should be 20% of the data 
X = df.drop(["ID","cluster","Category"],axis=1).values
Y = df[["Category"]].values

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y)

## 2. Building the model and training and evaluate the performance: 

In [ ]:
# To-do build the model and train it 
# note that you will be providing explanation about the hyper parameter tuning 
# So you will be iterating a number of times before getting the desired performance 

#### Hyper-parameter Tuning using GridSearch

In [70]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [71]:
def gridSearch(X_train,Y_train,tuned_parameters,metric=['accuracy','precision_weighted']):
    print(f'Fine-Tuning parameter for metric : {metric}')
    
    clf = GridSearchCV(RandomForestClassifier(),param_grid=tuned_parameters,scoring=metric, refit='accuracy',error_score="raise",n_jobs=2,return_train_score=True,verbose=5)
    results = clf.fit(X_train,Y_train)
    
    print(f'Best score : {results.best_score_}, using parameters : {results.best_params_}')
    
    return clf.cv_results_

We will be fine tuning the following parameters :
- n_estimators : which is the number of trees in the forest
- criterion : which is the selection measure to split a node in the decision tree
- max_features : the number of features to consider when looking for the best split. either (sqrt(nb_features) or log(nb_features))

using the gridSearch algorithme

In [72]:
tuned_params = [{
        "n_estimators":range(50,151),
        "criterion": ["gini", "entropy"],
        "max_features":["sqrt","log2"]
                     }]
scores = ["precision","accuracy"]

In [74]:
import warnings

warnings.filterwarnings("ignore")

results = gridSearch(X_train,Y_train,tuned_params)

Fine-Tuning parameter for metric : ['accuracy', 'precision_weighted']
Fitting 5 folds for each of 404 candidates, totalling 2020 fits
Best score : 0.9349412492269635, using parameters : {'criterion': 'gini', 'max_features': 'sqrt', 'n_estimators': 79}


In [75]:
model = RandomForestClassifier(n_estimators=79,criterion='gini',max_features="sqrt")
fitted_estim = model.fit(X_train,Y_train)

y_hat = fitted_estim.predict(X_test)

In [76]:
# To-do : evaluate the model in terms of accuracy and precision 
# Provide evidence that your model is not overfitting 
from sklearn.metrics import precision_score, accuracy_score

y_hat_train = model.predict(X_train)

print(f"Accuracy on Training set : {accuracy_score(Y_train,y_hat_train)}")
print(f"Precision on Training set : {precision_score(Y_train,y_hat_train,average='weighted')}")

print(f"Accuracy on Test set : {accuracy_score(Y_test,y_hat)} ")
print(f"Precision on Test set : {precision_score(Y_test,y_hat,average='weighted')}")

Accuracy on Training set : 1.0
Precision on Training set : 1.0
Accuracy on Test set : 0.926829268292683 
Precision on Test set : 0.9244641537324464


The model performed well also in the test set so there is no overfitting

In [77]:
from sklearn.metrics import classification_report

print(f'Classification report on test set : \n{classification_report(Y_test,y_hat)}')

Classification report on test set : 
              precision    recall  f1-score   support

         0.0       0.96      0.99      0.98       107
         1.0       0.00      0.00      0.00         1
         2.0       1.00      0.40      0.57         5
         3.0       0.40      0.50      0.44         4
         4.0       0.67      0.67      0.67         6

    accuracy                           0.93       123
   macro avg       0.61      0.51      0.53       123
weighted avg       0.92      0.93      0.92       123



> Hint : A Perfect accuracy on the train set suggest that we have an overfitted model So the student should be able to provide a detailed table about the hyper parameters / parameters tuning with a good conclusion stating that the model has at least an accuracy of 86% on the test set without signs of overfitting  

## 3. Create the summary test set with the additional predicted class column: 
In this part you need to add the predicted class as a column to your test dataframe and save this one 

In [87]:
# To-Do : create the complete test dataframe : it should contain all the feature column + the actual target and the ID as well  

test_df = pd.DataFrame(X_test,columns = df.columns[2:14])
test_df["Category"] = Y_test

# do the intersection between the test dataframe and the original dataframe to get the correct ID values
test_df = pd.merge(df,test_df,how="inner")

In [88]:
test_df.head()


,ID,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT,cluster
0,3,0.0,32,1.0,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3,3
1,9,0.0,32,1.0,50.9,65.5,23.2,21.2,6.9,8.69,4.10,83.0,13.7,71.3,3
2,13,0.0,33,1.0,36.3,78.6,23.6,22.0,7.0,8.56,5.38,78.0,19.4,68.7,3
3,21,0.0,33,1.0,44.3,49.8,32.1,21.6,13.1,7.44,5.59,103.0,30.2,74.0,3
4,25,0.0,34,1.0,40.5,32.4,29.6,27.1,5.8,10.50,4.56,91.0,26.6,72.0,3


In [94]:
# To-Do : Add the predicted_class column 
test_df["Predicted_class"] = y_hat

In [95]:
test_df.head()

,ID,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT,cluster,Predicted_class
0,3,0.0,32,1.0,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3,3,0.0
1,9,0.0,32,1.0,50.9,65.5,23.2,21.2,6.9,8.69,4.10,83.0,13.7,71.3,3,0.0
2,13,0.0,33,1.0,36.3,78.6,23.6,22.0,7.0,8.56,5.38,78.0,19.4,68.7,3,0.0
3,21,0.0,33,1.0,44.3,49.8,32.1,21.6,13.1,7.44,5.59,103.0,30.2,74.0,3,0.0
4,25,0.0,34,1.0,40.5,32.4,29.6,27.1,5.8,10.50,4.56,91.0,26.6,72.0,3,0.0


> Make sure you have 16 column in this test set  

In [97]:
# Save the test set 
test_df.to_csv("outputs/test_summary.csv",index=False)